In [1]:
import os
import json
import numpy as np
from tensorflow.keras.preprocessing import image
from facenetlib.Facenet_standalone import InceptionResNetV1

In [2]:
model = InceptionResNetV1(dimension=512)
model.load_weights("facenetlib/facenet512_weights.h5")  # ubah path jika perlu
print("✅ Model dan bobot berhasil dimuat.")


✅ Model dan bobot berhasil dimuat.


In [6]:
import os
import json
import numpy as np
from tensorflow.keras.preprocessing import image
from facenetlib.Facenet_standalone import InceptionResNetV1
from facenetlib import preprocessing, image_utils

# === Load model Facenet512 ===
model = InceptionResNetV1(dimension=512)
model.load_weights("facenetlib/facenet512_weights.h5")  # sesuaikan path
print("✅ Model Facenet512 berhasil dimuat.")

# === Preprocess gambar untuk Facenet (versi DeepFace) ===
def preprocess_image(img_path):
    # Load gambar RGB
    img_rgb, _ = image_utils.load_image(img_path)
    
    # Resize ke target size Facenet
    img_resized = preprocessing.resize_image(img_rgb, target_size=(160, 160))
    
    # Normalisasi sesuai standar Facenet
    img_normalized = preprocessing.normalize_input(img_resized, normalization="base")
    
    # Pastikan shape input (1, 160, 160, 3)
    if img_normalized.ndim == 3:
        img_normalized = np.expand_dims(img_normalized, axis=0)
    elif img_normalized.ndim == 4 and img_normalized.shape[0] == 1:
        pass
    else:
        raise ValueError(f"Bentuk input tidak dikenali: {img_normalized.shape}")
    
    return img_normalized

# === Ekstrak embedding ===
def get_embedding(img_path):
    preprocessed = preprocess_image(img_path)
    embedding = model.predict(preprocessed, verbose=0)
    return embedding[0].astype(float).tolist()

# === Folder gambar ===
input_folder = "cropted_faces"  # ganti ke path folder kamu
output_json = "embeddings.json"

embeddings_dict = {}

# Loop semua file di folder
for filename in os.listdir(input_folder):
    if filename.lower().endswith((".png", ".jpg", ".jpeg")):
        img_path = os.path.join(input_folder, filename)
        print(f"📷 Memproses: {filename}")
        try:
            embeddings_dict[filename] = get_embedding(img_path)
        except Exception as e:
            print(f"⚠️ Gagal memproses {filename}: {e}")

# Simpan ke JSON
with open(output_json, "w") as f:
    json.dump(embeddings_dict, f, indent=2)

print(f"✅ Embeddings disimpan di {output_json}")


✅ Model Facenet512 berhasil dimuat.
📷 Memproses: 7f38b75bc0a1e7cc77324323e7_f-0.jpg
📷 Memproses: WhatsApp Image 2025-07-30 at 16.57.34_f-0.jpg
📷 Memproses: WhatsApp Image 2025-07-30 at 16.57.37_f-0.jpg
📷 Memproses: WhatsApp Image 2025-07-30 at 16.57.47_f-0.jpg
📷 Memproses: WhatsApp Image 2025-07-30 at 16.57.48_f-0.jpg
✅ Embeddings disimpan di embeddings.json
